<a href="https://colab.research.google.com/github/snassimr/CommonLit/blob/master/CommonLit_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
SYS_PROJECT_DIR = '/content/gdrive/MyDrive/Colab Notebooks/CommonLit'
SYS_INPUT_DIR   = '/content/gdrive/MyDrive/Colab Notebooks/CommonLit'
SYS_OUTPUT_DIR   = '/content/gdrive/MyDrive/Colab Notebooks/CommonLit'

In [ ]:
import os
import pandas as pd
import shutil

In [ ]:
# shutil.rmtree('test123')

# Explore environment

In [ ]:
# import psutil
# import platform
# uname = platform.uname()
# print(f"System: {uname.system}")  #Windows or Linux
# print(f"Node Name: {uname.node}") # System name
# print(f"Release: {uname.release}") # OS release version like  10(Windows) or 5.4.0-72-generic(linux)
# print(f"Version: {uname.version}")
# print(f"Machine: {uname.machine}")  # machine can be AMD64 or x86-64
# print(f"Processor: {uname.processor}") #  Intel64 Family 6 or x86_64
# print("Physical cores:", psutil.cpu_count(logical=False))
# print("Total cores:", psutil.cpu_count(logical=True))

# def get_size(bytes, suffix="B"):
#     """
#     Scale bytes to its proper format- KB, MB, GB, TB and PB
#     """
#     factor = 1024
#     for unit in ["", "K", "M", "G", "T", "P"]:
#         if bytes < factor:
#             return f"{bytes:.2f}{unit}{suffix}"
#         bytes /= factor

# print("Virtual memory")
# svmem = psutil.virtual_memory()
# print(f"Total: {get_size(svmem.total)}")
# print(f"Available: {get_size(svmem.available)}")
# print(f"Used: {get_size(svmem.used)}")

# Read Data

## Train Data

In [ ]:
prompts_train   = pd.read_csv(os.path.join(SYS_INPUT_DIR,'prompts_train.csv'))
summaries_train = pd.read_csv(os.path.join(SYS_INPUT_DIR,'summaries_train.csv'))

In [ ]:
print(f"Prompts train shape: {prompts_train.shape}")
display(prompts_train.head())
print(f"Summary train shape: {summaries_train.shape}")
display(summaries_train.head())

Prompts train shape: (4, 4)


,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


Summary train shape: (7165, 5)


,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


## Test Data

In [ ]:
prompts_test   = pd.read_csv(os.path.join(SYS_INPUT_DIR,'prompts_test.csv'))
summaries_test = pd.read_csv(os.path.join(SYS_INPUT_DIR,'summaries_test.csv'))

In [ ]:
print(f"Prompts test shape: {prompts_test.shape}")
display(prompts_test.head())
print(f"Summary test shape: {summaries_test.shape}")
display(summaries_test.head())

Prompts test shape: (2, 4)


,prompt_id,prompt_question,prompt_title,prompt_text
0,abc123,Summarize...,Example Title 1,Heading\nText...
1,def789,Summarize...,Example Title 2,Heading\nText...


Summary test shape: (4, 3)


,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4


# Set Seed

In [ ]:
# set random seed
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(seed=42)

# Model Config

## Prepare Deberta v3 model

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.3 MB/s eta 0:00:00


## Initialize Model_Config

In [ ]:
Model_Config = {
    'model_name' : 'debertav3base',
    'base_model' : {
      'root_dir'      : 'base_model',
      'model_name'    : 'debertav3base',
      'num_labels'    : 1,
      'problem_type'  : 'regression',
      'hidden_dropout_prob' : 0.005,
      'attention_probs_dropout_prob' : 0.005,
      'max_length' : 512,
      'save_steps' : 100
    },
    'model_root_dir': 'model',
    'random_seed' : 42,
    'n_splits'    : 4,
    'num_train_epochs' : 5,
    'learning_rate' : 1.5e-5,
    'batch_size' : 32,
    'weight_decay'  : 0.02
}

## Download and save base model

In [ ]:
# import transformers
# from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

# base_model = AutoModel.from_pretrained(f"microsoft/deberta-v3-base")
# base_model_tokenizer = AutoTokenizer.from_pretrained(f"microsoft/deberta-v3-base")
# base_model_config = AutoConfig.from_pretrained(f"microsoft/deberta-v3-base")
# base_model_content = AutoModelForSequenceClassification.from_pretrained(f"microsoft/deberta-v3-base", config = base_model_config)

# import os
# base_model_root_dir = Model_Config['base_model']['root_dir']
# base_model_save_directory = os.path.join(SYS_PROJECT_DIR, base_model_root_dir)

# # Save model
# base_model.save_pretrained(base_model_save_directory)
# # Save tokenizer
# base_model_tokenizer.save_pretrained(base_model_save_directory)
# # Save config
# base_model_config.save_pretrained(base_model_save_directory)

## Load base model

In [ ]:
import os
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

base_model_root_dir = Model_Config['base_model']['root_dir']
base_model_save_directory = os.path.join(SYS_INPUT_DIR, base_model_root_dir)

base_model = AutoModel.from_pretrained(base_model_save_directory)
base_model_tokenizer = AutoTokenizer.from_pretrained(base_model_save_directory)
base_model_config  = AutoConfig.from_pretrained(base_model_save_directory)


## Update base model

In [ ]:
base_model_config.update(
    Model_Config['base_model']
)

## Update Model_Config

In [ ]:
base_model_content = AutoModelForSequenceClassification.from_pretrained(base_model_save_directory, config = base_model_config)
Model_Config['base_model']['base_model'] = base_model
Model_Config['base_model']['tokenizer']  = base_model_tokenizer
Model_Config['base_model']['config']     = base_model_config
Model_Config['base_model']['model_content'] = base_model_content

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /content/gdrive/MyDrive/Colab Notebooks/CommonLit/base_model and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
################################################################

# Text Feature Engineering

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import spacy
import re
from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import os
pyspellchecker_path = os.path.join(SYS_PROJECT_DIR, "pyspellchecker-0.7.2-py3-none-any.whl")
!pip install "{pyspellchecker_path}"
from spellchecker import SpellChecker

Processing ./gdrive/MyDrive/Colab Notebooks/CommonLit/pyspellchecker-0.7.2-py3-none-any.whl


In [ ]:
class Preprocessor:
    def __init__(self,
                tokenizer: AutoTokenizer,
                ) -> None:
        self.tokenizer  = tokenizer
        self.STOP_WORDS = set(stopwords.words('english'))

        self.spacy_ner_model = spacy.load('en_core_web_sm',)
        self.speller = SpellChecker() #Speller(lang='en')

    def text_length(self, df: pd.DataFrame, col:str) -> pd.Series:
        """ text length """
        tokenizer=self.tokenizer
        return df[col].progress_apply(lambda x: len(tokenizer.encode(x)))

    def word_overlap_length(self, row):
        """ word overlap length between prompt_text and text """
        def check_is_stop_word(word):
            return word in self.STOP_WORDS

        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))

    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int):
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)

        # # Optionally, you can get the frequency of common n-grams for a more nuanced analysis
        # original_ngram_freq = Counter(ngrams(original_words, n))
        # summary_ngram_freq = Counter(ngrams(summary_words, n))
        # common_ngram_freq = {ngram: min(original_ngram_freq[ngram], summary_ngram_freq[ngram]) for ngram in common_ngrams}

        return len(common_ngrams)

    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def misspelling_count(self, text):

        wordlist=text.split()
        misspelling_count = len(list(self.speller.unknown(wordlist)))

        return misspelling_count

    def run(self,
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:

        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x),
                skip_special_tokens=True
            )
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x),
                skip_special_tokens=True
            )

        )
        summaries["splling_err_num"] = summaries["text"].progress_apply(self.misspelling_count)

        # merge prompts and summaries
        input_df = summaries.merge(prompts, how="left", on="prompt_id")

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']

        input_df['word_overlap_count'] = input_df.progress_apply(self.word_overlap_length, axis=1)
        input_df['bigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1
        )
        input_df['trigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )

        input_df['quotes_count'] = input_df.progress_apply(self.quotes_count, axis=1)

        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])

In [ ]:
preprocessor = Preprocessor(tokenizer = base_model_tokenizer)

train = preprocessor.run(prompts_train, summaries_train, mode = "train")
test = preprocessor.run(prompts_test, summaries_test, mode = "test")

100%|██████████| 4/4 [00:00<00:00, 2422.35it/s]


# Modeling

## Create folds

In [ ]:
from sklearn.model_selection import KFold, GroupKFold

gkf = GroupKFold(n_splits = Model_Config['n_splits'])

for i, (_, val_index) in enumerate(gkf.split(train, groups=train["prompt_id"])):
    train.loc[val_index, "fold"] = i

## Define evaluation metrics

In [ ]:
def compute_metrics(eval_pred):

    """
    Calculates rmse as a evaluation metric to extend Trainer logging with additional metrics along with loss
    """

    from sklearn.metrics import mean_squared_error

    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """

    import numpy as np

    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):

    from sklearn.metrics import mean_squared_error

    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)

    return (content_score + wording_score)/2

## Model (regression)

## class Model

In [ ]:
class Model:
    def __init__(self,
                model_config : dict,
                model_dir: str,
                target: str
                ):
        self.inputs = ["prompt_text", "prompt_title", "prompt_question", "text"]
        self.input_col = "input"

        self.text_cols = [self.input_col]
        self.target = target
        self.target_cols = [target]

        self.model_config = model_config
        self.model_name = model_config['model_name']
        self.model_dir = model_dir

        self.base_tokenizer    = model_config['base_model']['tokenizer']
        self.base_model_config = model_config['base_model']['config']

        self.max_length = model_config['base_model']['max_length']

        seed_everything(seed=42)

        from transformers import DataCollatorWithPadding

        self.data_collator = DataCollatorWithPadding(
            tokenizer=self.base_tokenizer
        )

    def train_tokenize(self, examples: pd.DataFrame):
      labels = [examples[self.target]]
      tokenized = self.base_tokenizer(examples[self.input_col],
                      padding=False,
                      truncation=True,
                      max_length=self.max_length)
      return {
            **tokenized,
            "labels": labels,
      }

    def test_tokenize(self, examples: pd.DataFrame):
        tokenized = self.base_tokenizer(examples[self.input_col],
                        padding=False,
                        truncation=True,
                        max_length=self.max_length)
        return tokenized


    def train(self,
            fold: int,
            train_df: pd.DataFrame,
            valid_df: pd.DataFrame,
            model_config : dict
        ) -> None:
        """fine-tuning"""

        import pandas as pd
        from datasets import Dataset, load_dataset, load_from_disk
        from transformers import TrainingArguments, Trainer

        num_train_epochs =  model_config['num_train_epochs']
        learning_rate    =  model_config['learning_rate']
        batch_size       =  model_config['batch_size']
        weight_decay     =  model_config['weight_decay']
        save_steps       =  model_config['base_model']['save_steps']

        sep = self.base_tokenizer.sep_token

        train_df[self.input_col] = (
                    train_df["prompt_title"] + sep
                    + train_df["prompt_question"] + sep
                    + train_df["text"]
                  )

        valid_df[self.input_col] = (
                    valid_df["prompt_title"] + sep
                    + valid_df["prompt_question"] + sep
                    + valid_df["text"]
                  )

        train_df = train_df[[self.input_col] + self.target_cols]
        valid_df = valid_df[[self.input_col] + self.target_cols]

        train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
        val_dataset = Dataset.from_pandas(valid_df, preserve_index=False)

        train_tokenized_datasets = train_dataset.map(self.train_tokenize, batched=False)
        val_tokenized_datasets = val_dataset.map(self.train_tokenize, batched=False)

        model_content = model_config['base_model']['model_content']

        # eg. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold))

        # https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments

        training_args = TrainingArguments(
            output_dir=model_fold_dir,
            load_best_model_at_end=True, # Whether or not to load the best model found during training at the end of training
            optim  = 'adamw_torch',
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size, # The batch size per GPU/TPU core/CPU for training
            per_device_eval_batch_size=8,           # The batch size per GPU/TPU core/CPU for evaluation
            num_train_epochs=num_train_epochs,
            weight_decay=weight_decay,
            report_to='none',
            greater_is_better=False,
            save_strategy="steps",
            evaluation_strategy="steps", # "steps": Evaluation is done (and logged) every eval_steps.
            eval_steps=save_steps,
            save_steps=save_steps, # Number of updates steps before two checkpoint saves if save_strategy="steps"
            metric_for_best_model="rmse", # Use in conjunction with load_best_model_at_end to specify the metric to use to compare two different models. Default is loss
            save_total_limit=1
        )

        trainer = Trainer(
            model=model_content,
            args=training_args,
            train_dataset=train_tokenized_datasets,
            eval_dataset=val_tokenized_datasets,
            tokenizer=self.base_tokenizer,
            compute_metrics=compute_metrics,
            data_collator=self.data_collator
        )

        trainer.train()

        model_content.save_pretrained(self.model_dir)
        self.base_tokenizer.save_pretrained(self.model_dir)

    def predict(self,
                test_df: pd.DataFrame,
                fold: int,
                ):
        """predict content score"""

        from datasets import Dataset, load_dataset, load_from_disk
        from transformers import TrainingArguments, Trainer

        sep = self.base_tokenizer.sep_token

        in_text = (
                    test_df["prompt_title"] + sep
                    + test_df["prompt_question"] + sep
                    + test_df["text"]
                  )
        test_df[self.input_col] = in_text

        test_ = test_df[[self.input_col]]

        test_dataset = Dataset.from_pandas(test_, preserve_index=False)
        test_tokenized_dataset = test_dataset.map(self.test_tokenize, batched=False)

        model_content = AutoModelForSequenceClassification.from_pretrained(f"{self.model_dir}")
        model_content.eval()

        # e.g. "bert/fold_0/"
        model_fold_dir = os.path.join(self.model_dir, str(fold))

        test_args = TrainingArguments(
            output_dir=self.model_dir,
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = 4,
            dataloader_drop_last = False,
        )

        # init trainer
        infer_content = Trainer(
                      model = model_content,
                      tokenizer=self.base_tokenizer,
                      data_collator=self.data_collator,
                      args = test_args)

        preds = infer_content.predict(test_tokenized_dataset)[0]

        return preds

## ############################

## train_by_fold

In [ ]:
def train_by_fold(
        train_df: pd.DataFrame,
        target:str,
        model_config: dict
    ):

    model_root_dir = model_config['model_root_dir']
    model_name = model_config['model_name']
    n_splits   = model_config['n_splits']

    for fold in range(Model_Config['n_splits']):
        print(f"fold {fold}:")

        train_data = train_df[train_df["fold"] != fold]
        valid_data = train_df[train_df["fold"] == fold]

        model_dir =  os.path.join(SYS_PROJECT_DIR, model_root_dir, f"{target}/{model_name}/fold_{fold}")

        if os.path.exists(model_dir):
          shutil.rmtree(model_dir)

        model = Model(
            model_config=model_config,
            model_dir = model_dir,
            target=target
           )

        model.train(
            fold=fold,
            train_df=train_data,
            valid_df=valid_data,
            model_config = model_config
        )

## Run train_by_fold

In [ ]:
for target in ["content", "wording"]:
    train_by_fold(
        train.head(500),
        target=target,
        model_config = Model_Config
    )

fold 0:


<ipython-input-23-6dc40e8184f0>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[self.input_col] = (
<ipython-input-23-6dc40e8184f0>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[self.input_col] = (


Map:   0%|          | 0/343 [00:00<?, ? examples/s]

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

fold 3:


<ipython-input-23-6dc40e8184f0>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[self.input_col] = (
<ipython-input-23-6dc40e8184f0>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df[self.input_col] = (


Map:   0%|          | 0/428 [00:00<?, ? examples/s]

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Rmse
100,No log,0.818005,0.904436
200,No log,0.629663,0.793513
300,No log,0.553984,0.744301
400,No log,0.537891,0.733410
500,0.334800,0.555668,0.745432


# Evaluation

In [ ]:
def predict_train(
    train_df: pd.DataFrame,
    target:str,
    model_config:dict
    ) -> pd.DataFrame:
    """predict oof data"""

    model_root_dir = model_config['model_root_dir']
    model_name = model_config['model_name']
    n_splits = model_config['n_splits']

    for fold in range(n_splits):

        print(f"fold {fold}:")

        valid_data = train_df[train_df["fold"] == fold]

        model_dir =  os.path.join(SYS_PROJECT_DIR, model_root_dir, f"{target}/{model_name}/fold_{fold}")

        model = Model(
            model_config = model_config,
            model_dir = model_dir,
            target=target
           )

        pred = model.predict(
            test_df=valid_data,
            fold=fold
        )

        train_df.loc[valid_data.index, f"{target}_pred"] = pred

    return train_df

In [ ]:
for target in ["content", "wording"]:

  from sklearn.metrics import mean_squared_error

  train = predict_train(
        train.head(500),
        target=target,
        model_config = Model_Config
    )
  rmse = mean_squared_error(train[target], train[f"{target}_pred"], squared=False)
  print(f"cv {target} rmse: {rmse}")

fold 0:


<ipython-input-25-7725510204d3>:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[self.input_col] = in_text


Map:   0%|          | 0/157 [00:00<?, ? examples/s]

fold 1:


<ipython-input-25-7725510204d3>:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[self.input_col] = in_text


...

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

fold 3:


<ipython-input-25-7725510204d3>:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[self.input_col] = in_text


Map:   0%|          | 0/72 [00:00<?, ? examples/s]

cv wording rmse: 0.6571001653074217


# Prediction

In [ ]:
def predict_test(
      test_df: pd.DataFrame,
      target:str,
      model_config:dict
    ) -> pd.DataFrame:
    """predict using mean folds"""

    model_root_dir = model_config['model_root_dir']
    model_name = model_config['model_name']
    n_splits = model_config['n_splits']

    for fold in range(n_splits):
        print(f"fold {fold}:")

        model_dir =  os.path.join(SYS_PROJECT_DIR, model_root_dir, f"{target}/{model_name}/fold_{fold}")

        model = Model(
            model_config = model_config,
            model_dir = model_dir,
            target=target
           )

        pred = model.predict(
            test_df=test_df,
            fold=fold
        )

        test_df[f"{target}_pred_{fold}"] = pred

    test_df[f"{target}"] = test_df[[f"{target}_pred_{fold}" for fold in range(n_splits)]].mean(axis=1)

    return test_df

In [ ]:
for target in ["content", "wording"]:
    test = predict_test(
          test,
          target=target,
          model_config = Model_Config
      )

fold 0:


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

fold 1:


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

fold 2:


...

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

fold 2:


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

fold 3:


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

# LGBM

## Data Preparation

In [ ]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text",
                "prompt_question", "prompt_title",
                "prompt_text"
               ] + targets

## Modeling

In [ ]:
import lightgbm as lgb

n_splits = Model_Config['n_splits']

model_dict = {}

for target in targets:
    models = []

    for fold in range(n_splits):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
                  'boosting_type': 'gbdt', # gbdt
                  'random_state': 42,
                  'objective': 'regression',
                  'metric': 'rmse',
                  'learning_rate': 0.05,
                  'min_child_samples' : 10
                  }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=1000,
                          #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=10, verbose=True),
                              lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        models.append(model)

    model_dict[target] = models

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 564
[LightGBM] [Info] Number of data points in the train set: 343, number of used features: 10
[LightGBM] [Info] Start training from score 0.049033
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

## Evaluation (CV)

In [ ]:
import numpy as np

rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []

    for fold, model in enumerate(models):
        # ilocで取り出す行を指定
        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)

    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.4579184830838203
wording_rmse : 0.6515022903130216
mcrmse : 0.554710386698421


# Prediction

In [ ]:
n_splits = Model_Config['n_splits']

drop_columns = [
                #"fold",
                "student_id", "prompt_id", "text",
                "prompt_question", "prompt_title",
                "prompt_text",
                "input"
               ] + [
                f"content_pred_{i}" for i in range(n_splits)
                ] + [
                f"wording_pred_{i}" for i in range(n_splits)
                ]

In [ ]:
pred_dict = {}
for target in targets:
    models = model_dict[target]
    preds = []

    for fold, model in enumerate(models):
        X_eval_cv = test.drop(columns=drop_columns)

        pred = model.predict(X_eval_cv)
        preds.append(pred)

    pred_dict[target] = preds

In [ ]:
for target in targets:
    preds = pred_dict[target]
    for i, pred in enumerate(preds):
        test[f"{target}_pred_{i}"] = pred

    test[target] = test[[f"{target}_pred_{fold}" for fold in range(n_splits)]].mean(axis=1)

In [ ]:
submission_file = os.path.join(SYS_OUTPUT_DIR, "submission.csv")
test[["student_id", "content", "wording"]].to_csv(submission_file, index=False)